In [2]:
import pandas as pd
import numpy as np

In [3]:
train_data=pd.read_csv('../Data/train_data.csv')
topic_dict=pd.read_csv('../Data/topic_dict.csv')

In [4]:
train_data.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


In [5]:
topic_dict

,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


In [6]:
train_data['topic_idx'].unique()

array([4, 6, 5, 0, 1, 3, 2], dtype=int64)

In [7]:
topic_dict['topic_idx'].unique()

array([0, 1, 2, 3, 4, 5, 6], dtype=int64)

- label은 총 0 ~ 6으로 7개를 가지고 있다.

In [8]:
train_data['topic_idx'].value_counts()

4    7629
2    7362
5    6933
6    6751
1    6222
3    5933
0    4824
Name: topic_idx, dtype: int64

- label은 대체로 골고루 존재한다.

In [9]:
# !pip install transformers==4.2.0 sentencepiece

In [10]:
import torch

from sklearn.metrics import accuracy_score
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, TrainingArguments, XLMRobertaConfig, Trainer
from sklearn.model_selection import train_test_split

In [11]:
data=pd.read_csv('../Data/train_data.csv')

In [12]:
data

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1
45652,45652,답변하는 배기동 국립중앙박물관장,2


In [13]:
data['title'].str.len().value_counts()

30    4284
29    4281
28    4192
27    4035
31    4007
26    3427
32    3421
25    2698
24    2268
33    2158
23    1706
34    1366
22    1312
21    1005
20     751
35     657
19     566
18     492
17     437
16     379
15     325
36     293
14     273
13     256
37     180
12     166
11     156
10     109
38      99
9       91
39      61
8       50
40      44
41      38
7       27
42      16
6        9
5        8
44       5
43       5
4        1
Name: title, dtype: int64

In [14]:
# load model and tokenizer
MODEL_NAME = "xlm-roberta-large"
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)

In [15]:
train_dataset,val_dataset = train_test_split(data,test_size=0.2,stratify=data['topic_idx'],random_state=42)

In [16]:
train_label=train_dataset['topic_idx'].values
val_label=val_dataset['topic_idx'].values

In [17]:
def tokenized_dataset(dataset, tokenizer):
    tokenized_sentences = tokenizer(
      list(dataset['title']),
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=44
      )
    return tokenized_sentences

In [18]:
# Dataset 구성.
class news_Dataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_dataset, labels):
        self.tokenized_dataset = tokenized_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_dataset.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
    def get_classes(self):
        return self.labels

In [19]:
# tokenizing dataset
tokenized_train = tokenized_dataset(train_dataset, tokenizer)
tokenized_val = tokenized_dataset(val_dataset, tokenizer)

In [20]:
# make dataset for pytorch.
news_train_dataset = news_Dataset(tokenized_train, train_label)
news_val_dataset = news_Dataset(tokenized_val, val_label)

In [21]:
news_train_dataset

In [22]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [23]:
# setting model hyperparameter
bert_config = XLMRobertaConfig.from_pretrained(MODEL_NAME)
bert_config.num_labels = 7
model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME, config=bert_config)
model.to(device)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

In [26]:
# 평가를 위한 metrics function.
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [25]:
class CustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        try:
            return super().find_class(__name__, name)
        except AttributeError:
            return super().find_class(module, name)

UsageError: %%writefile is a cell magic, but the cell body is empty.


In [27]:

training_args = TrainingArguments(
    output_dir='./results',
    save_total_limit=3,
    save_steps=3000,
    num_train_epochs=40,
    learning_rate=1e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=64,
    warmup_steps=5000,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=3000,
    evaluation_strategy='steps',
    eval_steps = 3000,
    dataloader_num_workers=1,
    label_smoothing_factor=0.2
  )

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=news_train_dataset,
  eval_dataset=news_val_dataset,
  compute_metrics=compute_metrics
)

# train model
trainer.train()

UsageError: the following arguments are required: filename


# inference

In [52]:
test_data

,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
...,...,...
9126,54780,인천 오후 3시35분 대설주의보…눈 3.1cm 쌓여
9127,54781,노래방에서 지인 성추행 외교부 사무관 불구속 입건종합
9128,54782,40년 전 부마항쟁 부산 시위 사진 2점 최초 공개
9129,54783,게시판 아리랑TV 아프리카개발은행 총회 개회식 생중계


In [8]:
test_data=pd.read_csv('./data/test_data.csv')
submission=pd.read_csv('./data/sample_submission.csv')

In [16]:
# roBERTa model
TOK_NAME="xlm-roberta-large"
tokenizer = XLMRobertaTokenizer.from_pretrained(TOK_NAME)

MODEL_NAME='./results/checkpoint-45000'
model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

In [20]:
# 임시로 label 사용
test_label=submission['topic_idx'].values

tokenized_test = tokenized_dataset(test_data, tokenizer)
news_test_dataset = news_Dataset(tokenized_test, test_label)

In [28]:
from torch.utils.data import DataLoader
def inference(model, tokenized_sent, device):
    dataloader = DataLoader(tokenized_sent, batch_size=40, shuffle=False)
    model.eval()
    output_pred = []
  
    for i, data in enumerate(dataloader):
        with torch.no_grad():
            outputs = model(
              input_ids=data['input_ids'].to(device),
              attention_mask=data['attention_mask'].to(device),
              #token_type_ids=data['token_type_ids'].to(device)
              )
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            result = np.argmax(logits, axis=-1)

            output_pred.extend(result)
    return output_pred
    #return np.array(output_pred).flatten()

In [29]:
pred_answer = inference(model, news_test_dataset, device)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [32]:
submission['topic_idx']=pred_answer
submission.to_csv('./submission/roberta_base.csv',index=False)

In [33]:
submission

,index,topic_idx
0,45654,0
1,45655,3
2,45656,2
3,45657,0
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,3
9129,54783,2
